In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv
/kaggle/input/riiid-test-answer-prediction/example_test.csv
/kaggle/input/riiid-test-answer-prediction/questions.csv
/kaggle/input/riiid-test-answer-prediction/train.csv
/kaggle/input/riiid-test-answer-prediction/lectures.csv
/kaggle/input/riiid-test-answer-prediction/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/riiid-test-answer-prediction/riiideducation/__init__.py


In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import riiideducation

In [3]:
types = {
        'row_id': 'int64', 
        'timestamp': 'int64', 
        'user_id': 'int32', 
        'content_id': 'int16', 
        'content_type_id': 'int8',
        'task_container_id': 'int16', 
        'user_answer': 'int8', 
        'answered_correctly': 'int64', 
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'boolean'
}

In [4]:
train = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv', 
    low_memory=False, 
    nrows=10**7, 
    dtype=types
)
questions=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',
                      dtype={'question_id': 'int16','part': 'int8','bundle_id': 'int16','tags': 'str'})
lectures=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv',
                    dtype={'lectures_id': 'int16','tag': 'int16','part': 'int8','type_of': 'str'})
ex_test=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')

train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,<NA>
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


**train**

row_id: (int64) ID code for the row.

timestamp: (int64) the time in milliseconds between this user interaction and the first event completion from that user.

user_id: (int32) ID code for the user.

content_id: (int16) ID code for the user interaction

content_type_id: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

task_container_id: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

user_answer: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

answered_correctly: (int8) if the user responded correctly. Read -1 as null, for lectures.

prior_question_elapsed_time: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

prior_question_had_explanation: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.

In [5]:
questions.tail()

,question_id,bundle_id,correct_answer,part,tags
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125
13522,13522,13522,3,5,55


**QUESTIONS.CSV**

question_id = content_id: foreign key for the train/test content_id column, when the content type is question (0).

bundle_id: code for which questions are served together.

correct_answer: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

part: top level category code for the question.

tags: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

In [6]:
lectures.head(10)

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question
5,335,114,2,concept
6,484,179,5,concept
7,641,134,6,solving question
8,761,93,1,concept
9,814,80,5,solving question


lectures.csv: metadata for the lectures watched by users as they progress in their education.

lecture_id: foreign key for the train/test content_id column, when the content type is lecture (1).

part: top level category code for the lecture.

tag: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

type_of: brief description of the core purpose of the lecture

In [7]:
train.head()


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,<NA>
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [8]:
ex_test.head(50)

,row_id,group_num,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
0,0,0,0,275030867,5729,0,0,NaN,NaN,[],[]
1,1,0,13309898705,554169193,12010,0,4427,19000.0,True,NaN,NaN
2,2,0,4213672059,1720860329,457,0,240,17000.0,True,NaN,NaN
3,3,0,62798072960,288641214,13262,0,266,23000.0,True,NaN,NaN
4,4,0,10585422061,1728340777,6119,0,162,72400.0,True,NaN,NaN
5,5,0,18020362258,1364159702,12023,0,4424,18000.0,True,NaN,NaN
6,6,0,2325432079,1521618396,574,0,1367,18000.0,True,NaN,NaN
7,7,0,39456940781,1317245193,12043,0,5314,17000.0,True,NaN,NaN
8,8,0,3460555189,1700555100,7910,0,532,21000.0,True,NaN,NaN
9,9,0,2214770464,998511398,7908,0,393,21000.0,True,NaN,NaN


example_test_rows.csv Three sample groups of the test set data as it will be delivered by the time-series API. The format is largely the same as train.csv. There are two different columns that mirror what information the AI tutor actually has available at any given time, but with the user interactions grouped together for the sake of API performance rather than strictly showing information for a single user at a time. Some users will appear in the hidden test set that have NOT been presented in the train set, emulating the challenge of quickly adapting to modeling new arrivals to a website.

prior_group_responses (string) provides all of the user_answer entries for previous group in a string representation of a list in the first row of the group. All other rows in each group are null. If you are using Python, you will likely want to call eval on the non-null rows. Some rows may be null, or empty lists.

prior_group_answers_correct (string) provides all the answered_correctly field for previous group, with the same format and caveats as prior_group_responses. Some rows may be null, or empty lists.

**Какие признаки нам помогут для предсказаний?**

## Предварительные идеи:

1) запоминаем историю ответов пользователя ( долю правильных ответов конкретного пользователя = accuracy)

2) запоминаем историю овтетов всех пользователей на конкретные вопросы( долю правильных ответов на конкретный вопрос)

????3) запоминаем историю ответов для блоков вопросов ( но нужно ли? мб 2 уже включает это)

4) смотеть, просмотрены ли лекции по материалам вопроса( через скалярное произведение tag question на tag просмотренных лекций, делёное на кол-во tag вопроса)

5??5-6) запоминаем среднее время на правильный и/или неправильный ответ для конкретного пользователя(prior_question_elapsed_time делаем lag -1) или запоминаем просто среднее время
или только ср вр на правильный ответ

7??7-8) запоминаем среднее время на правильный и/или неправильный ответ для конкретного вопроса  
или запоминаем просто среднее время или только ср вр на правильный ответ

9+) запоминаем accuracy ответов для каждой из групп вопросов, полученных после класстеризации вопросов по tag questions

10+)  запоминаем среднее время ответов для каждой из групп вопросов, полученных после класстеризации вопросов по tag questions







## FEATURE ENGINEERING and DATA CLEANING

In [9]:
#prior_question_elapsed_time отвечает за время ответа на предыдущий вопрос, сдвинув, получим время ответа на текущий вопрос
train['question_elapsed_time'] = train['prior_question_elapsed_time'].shift(-1)
#train['prior_question_elapsed_time'].fillna(train['prior_question_elapsed_time'].mean(), inplace = True)
train['question_had_explanation'] = train['prior_question_had_explanation'].shift(-1)
train['question_had_explanation'].fillna(value=False, inplace = True)
train.replace(False, 0, inplace=True)
train.replace(True, 1, inplace=True)


train

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_elapsed_time,question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,<NA>,37000.0,False
1,1,56943,115,5716,0,2,2,1,37000.0,False,55000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False,19000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False,11000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False,5000.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,9999995,646228695,216747867,8805,0,410,1,1,23000.0,True,13000.0,True
9999996,9999996,646321314,216747867,5708,0,411,0,0,13000.0,True,66000.0,True
9999997,9999997,646393443,216747867,5434,0,412,0,0,66000.0,True,37000.0,True
9999998,9999998,646445632,216747867,6429,0,413,3,1,37000.0,True,16000.0,True


In [10]:
#Убираем лекции и группируем по user_id
train_questions_only_df = train[train['answered_correctly']!=-1]
grouped_by_user_df = train_questions_only_df.groupby('user_id')


user_ans_df = grouped_by_user_df.agg(
    {
        'answered_correctly': [
            'mean', 
            'count']}
).copy()

user_ans_df.columns = [
    'mean_user_accuracy',
    'questions_answered'
]

In [11]:
user_ans_df

,mean_user_accuracy,questions_answered
user_id,,
115,0.695652,46
124,0.233333,30
2746,0.578947,19
5382,0.672000,125
8623,0.642202,109
...,...,...
216740611,0.685714,210
216741934,0.788462,104
216743480,0.294118,17


In [12]:
##пока что просто среднее время на ответ без разгроничения на правильность
user_ans_time_df = grouped_by_user_df.agg(
    {
        'prior_question_elapsed_time': [
            'mean']}
).copy()

user_ans_time_df.columns = [
    'mean_user_ans_time'
]

user_ans_time_df

,mean_user_ans_time
user_id,
115,19933.310547
124,18793.000000
2746,18055.554688
5382,36048.386719
8623,26107.408203
...,...
216740611,21446.085938
216741934,19378.630859
216743480,28750.000000


In [13]:
#Группируем по content_id
grouped_by_content_df = train_questions_only_df.groupby('content_id')
content_answers_df = grouped_by_content_df.agg(
    {
        'answered_correctly': [
            'mean', 
            'count'
        ]
    }
).copy()

content_answers_df.columns = [
    'mean_que_accuracy', 
    'question_was_asked'
]

content_answers_df


,mean_que_accuracy,question_was_asked
content_id,,
0,0.908828,691
1,0.893939,726
2,0.562319,4501
3,0.780316,2276
4,0.627973,3153
...,...,...
13518,0.790698,86
13519,0.581395,86
13520,0.682927,82


In [14]:
que_ans_time_df = grouped_by_content_df.agg(
    {
        'question_elapsed_time': [
            'mean']}
).copy()

que_ans_time_df.columns = [
    'mean_que_ans_time'
]

que_ans_time_df

,mean_que_ans_time
content_id,
0,19923.191406
1,19569.638672
2,24637.886719
3,21794.638672
4,22014.281250
...,...
13518,18614.457031
13519,30108.433594
13520,31182.927734


In [15]:
# Убираем неправильные ответы для того, чтобы посчитать среднее время правильных ответов на вопросы
train_questions_only_well = train[train['answered_correctly']!=0]

In [16]:
#Подсчёт времени правильных ответов на вопросы
grouped_by_questions_well = train_questions_only_well.groupby('content_id')
quest_well_df = grouped_by_questions_well.agg(
    {
    'question_elapsed_time':['mean']
        #'prior_question_had_explanation':['sum','count']
        
    }).copy()
quest_well_df.columns = [

    'mean_question_elapsed_time'
   # 'question_had_explanation_true_count',
    #'question_had_explanation_all_count'
]

quest_well_df["mean_question_elapsed_time"] = quest_well_df["mean_question_elapsed_time"].astype(int)
#quest_well_df['question_had_explanation_false_count'] = quest_well_df['question_had_explanation_all_count'] - quest_well_df['question_had_explanation_true_count']

quest_well_df

#Закомментированные фичи плохо повлияли на public score

,mean_question_elapsed_time
content_id,
0,19601
1,19279
2,23772
3,21292
4,21312
...,...
32491,28343
32535,23460
32570,31156


In [17]:
#Подсчёт времени правильных ответов на вопросы, группировка по user_id
grouped_by_user_well = train_questions_only_well.groupby('user_id')
user_well_df = grouped_by_user_well.agg(
    {
    'question_elapsed_time':['mean']
    }).copy()
user_well_df.columns = [

    'mean_question_elapsed_time_user']
user_well_df.fillna(max)
#user_well_df["mean_prior_question_elapsed_time_user"] = user_well_df["mean_prior_question_elapsed_time_user"].astype(int)


user_well_df

,mean_question_elapsed_time_user
user_id,
115,20258.064453
124,21285.572266
2746,16727.273438
5382,33141.175781
8623,21490.140625
...,...
216740611,21157.853516
216741934,18658.523438
216743480,20250.000000


## Получили фичи 1-2, 5-8. Далее можно улучшить 5-8 и заняться кластеризацией для фичей 9+


In [18]:
#Удаляем пропуски
questions = questions.dropna()
questions

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125


In [19]:
import re
import sys

In [20]:
s = set()
dirt_list = questions["tags"].str.split(" ", expand = True) 
for merge in dirt_list:
    ns = set(dirt_list[merge])
    s.update(ns)
   

In [21]:
s = {i for i in s if pd.notna(i)}
s

{'0',
 '1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '16',
 '160',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '184',
 '185',
 '186',
 '187',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',


In [22]:
#Преобразование последовательностей тегов в бинарные вектора
buf = []
tagg_df = pd.DataFrame(index=questions.value_counts())
tagg_df.index = [i for i in range(13522)]

f = questions['tags']
for i in s:
    buf = []
    for j in range(len(f.values)):
        d = f.values[j].split(' ')
        #print(d)
        if i in d:
            buf.append(1)
        else:
            buf.append(0)
    df = pd.DataFrame(buf, columns = [i])
    #print(df)
    tagg_df = tagg_df.join(df)
tagg_df     
    


,38,178,2,81,128,187,23,46,77,48,...,71,93,131,88,122,83,42,102,76,110
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
questions.index =  [i for i in range(13522)]
questions


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13517,13518,13518,3,5,14
13518,13519,13519,3,5,8
13519,13520,13520,2,5,73
13520,13521,13521,0,5,125


Была проведена кластеризация векторов тегов по алгоритму k-means, график показал плечо на N = 40

In [24]:
from gensim import corpora, models, matutils
from sklearn.cluster import KMeans

'''X = tagg_df
distorsions = []
# %%
for k in range(2, 150):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    distorsions.append(kmeans.inertia_)
# %%
fig = plt.figure(figsize=(10, 10))
plt.plot(range(2, 150), distorsions)
plt.grid(True)
plt.title('Elbow curve')
plt.show()'''
# %%
# оптимальное число кластеров = 40

kmeans = KMeans(n_clusters=40)
km = kmeans.fit_predict(tagg_df)
print(km)

# %%

tagg_df['tag_clust'] = km
print()


[22 29  3 ...  8  0  4]



In [25]:
tagg_df['index'] = tagg_df.index
questions['index'] = questions.index
questions = questions.merge(tagg_df, how='left', on = 'index')


In [26]:
questions

,question_id,bundle_id,correct_answer,part,tags,index,38,178,2,81,...,93,131,88,122,83,42,102,76,110,tag_clust
0,0,0,0,1,51 131 162 38,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,22
1,1,1,1,1,131 36 81,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,29
2,2,2,0,1,131 101 162 92,2,0,0,0,0,...,0,1,0,0,0,0,0,0,0,3
3,3,3,0,1,131 149 162 29,3,0,0,0,0,...,0,1,0,0,0,0,0,0,0,29
4,4,4,3,1,131 5 162 38,4,1,0,0,0,...,0,1,0,0,0,0,0,0,0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13517,13518,13518,3,5,14,13517,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
13518,13519,13519,3,5,8,13518,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
13519,13520,13520,2,5,73,13519,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
13520,13521,13521,0,5,125,13520,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
questions = questions.drop(columns = [i for i in list(questions.columns) if (i != 'question_id') & (i != 'tag_clust')])
questions

,question_id,tag_clust
0,0,22
1,1,29
2,2,3
3,3,29
4,4,22
...,...,...
13517,13518,25
13518,13519,6
13519,13520,8
13520,13521,0


In [28]:
questions.rename(columns=lambda x: x.replace('question_id', 'content_id'), inplace=True)
questions

,content_id,tag_clust
0,0,22
1,1,29
2,2,3
3,3,29
4,4,22
...,...,...
13517,13518,25
13518,13519,6
13519,13520,8
13520,13521,0


## Фича 4 (связь вопросов с лекциями)

In [29]:
grouped_by_container_df = train.groupby('task_container_id')
container_df = grouped_by_container_df.agg(
    {'answered_correctly': [
            'mean', 
            'count'],
    'content_type_id':['sum']
    }).copy()
container_df.columns = [
    'mean_container_acc', 
    'container_unit_count',
    'lecs_in_container']
container_df["lecs_in_container"] = container_df["lecs_in_container"].astype(int)
container_df["lecs_to_qs"] = container_df["lecs_in_container"]/container_df["container_unit_count"]

container_df


,mean_container_acc,container_unit_count,lecs_in_container,lecs_to_qs
task_container_id,,,,
0,0.680675,39690,7,0.000176
1,0.528523,39705,1,0.000025
2,0.436121,39645,4,0.000101
3,0.543857,39652,3,0.000076
4,0.481446,69258,4,0.000058
...,...,...,...,...
9995,0.666667,15,0,0.000000
9996,0.800000,15,1,0.066667
9997,0.769231,13,0,0.000000


## Далее подсоединяем к train сгенерированные фичи

In [30]:
target = 'answered_correctly'
train = train[train[target] != -1] #delet all lections
#merge all features in one df
train = train.merge(user_ans_df, how='left', on='user_id')
train = train.merge(content_answers_df, how='left', on='content_id')
train = train.merge(container_df, how='left', on='task_container_id')
train = train.merge(user_ans_time_df, how='left', on='user_id')
train = train.merge(que_ans_time_df, how='left', on='content_id')
train = train.merge(quest_well_df, how='left', on='content_id')
train = train.merge(user_well_df, how='left', on='user_id')
train = train.merge(questions, how='left', on = 'content_id')

train

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,...,question_was_asked,mean_container_acc,container_unit_count,lecs_in_container,lecs_to_qs,mean_user_ans_time,mean_que_ans_time,mean_question_elapsed_time,mean_question_elapsed_time_user,tag_clust
0,0,0,115,5692,0,1,3,1,NaN,<NA>,...,3548,0.528523,39705,1,0.000025,19933.310547,16801.380859,15473.0,20258.064453,0
1,1,56943,115,5716,0,2,2,1,37000.0,False,...,2376,0.436121,39645,4,0.000101,19933.310547,16428.082031,15253.0,20258.064453,0
2,2,118363,115,128,0,0,0,1,55000.0,False,...,1914,0.680675,39690,7,0.000176,19933.310547,18156.904297,17947.0,20258.064453,3
3,3,131167,115,7860,0,3,0,1,19000.0,False,...,2137,0.543857,39652,3,0.000076,19933.310547,20252.582031,20070.0,20258.064453,29
4,4,137965,115,7922,0,4,1,1,11000.0,False,...,1878,0.481446,69258,4,0.000058,19933.310547,17750.400391,17697.0,20258.064453,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9804599,9999995,646228695,216747867,8805,0,410,1,1,23000.0,True,...,624,0.632155,5940,156,0.026263,42221.257812,20012.945312,19949.0,42727.265625,0
9804600,9999996,646321314,216747867,5708,0,411,0,0,13000.0,True,...,5287,0.644307,5963,136,0.022807,42221.257812,47403.847656,42117.0,42727.265625,0
9804601,9999997,646393443,216747867,5434,0,412,0,0,66000.0,True,...,1809,0.639624,5855,148,0.025278,42221.257812,23305.748047,22033.0,42727.265625,0
9804602,9999998,646445632,216747867,6429,0,413,3,1,37000.0,True,...,3855,0.615425,5783,155,0.026803,42221.257812,25951.912109,23769.0,42727.265625,0


## Получили набор данных, теперь нужно его почистить от пропусков

In [31]:
train.isnull().sum()

row_id                                  0
timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
answered_correctly                      0
prior_question_elapsed_time         39388
prior_question_had_explanation      39388
question_elapsed_time              228579
question_had_explanation                0
mean_user_accuracy                      0
questions_answered                      0
mean_que_accuracy                       0
question_was_asked                      0
mean_container_acc                      0
container_unit_count                    0
lecs_in_container                       0
lecs_to_qs                              0
mean_user_ans_time                     12
mean_que_ans_time                       1
mean_question_elapsed_time              2
mean_question_elapsed_time_user   

In [32]:
train['prior_question_had_explanation'].fillna(value = False, inplace = True)
train = train.fillna(train.mean())
#train['prior_question_elapsed_time'].fillna(value = train['prior_question_elapsed_time'].mean(), inplace = True)
#train['prior_question_had_explanation'].fillna(value = False, inplace = True)
#train['question_elapsed_time'].fillna(value = train['question_elapsed_time'].mean(), inplace = True)
#train['mean_user_ans_time'].fillna(value = train['mean_user_ans_time'].mean(), inplace = True)
#train['mean_que_ans_time'].fillna(value = train['mean_que_ans_time'].mean(), inplace = True)
#train['mean_question_elapsed_time'].fillna(value = train['mean_question_elapsed_time'].mean(), inplace = True)
#train['mean_question_elapsed_time_user'].fillna(value = train['mean_question_elapsed_time_user'].mean(), inplace = True)
tr_mean = train.mean()
tr_mean

row_id                             5.000112e+06
timestamp                          7.543714e+09
user_id                            1.079965e+08
content_id                         4.981699e+03
content_type_id                    0.000000e+00
task_container_id                  8.810893e+02
user_answer                        1.423823e+00
answered_correctly                 6.586115e-01
prior_question_elapsed_time        2.531944e+04
prior_question_had_explanation     9.032037e-01
question_elapsed_time              2.535818e+04
question_had_explanation           8.847134e-01
mean_user_accuracy                 6.586115e-01
questions_answered                 2.330212e+03
mean_que_accuracy                  6.586115e-01
question_was_asked                 2.522532e+03
mean_container_acc                 6.260642e-01
container_unit_count               1.190190e+04
lecs_in_container                  1.556682e+02
lecs_to_qs                         1.947131e-02
mean_user_ans_time                 2.518

In [33]:
#Удаляем ненужные колонки
col_to_drop = [
                #'timestamp',
               'row_id',
               #'user_id',
               #'content_id',
               'content_type_id',
               #'task_container_id',
               'user_answer']
train=train.drop(columns = col_to_drop)

train

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_elapsed_time,question_had_explanation,mean_user_accuracy,...,question_was_asked,mean_container_acc,container_unit_count,lecs_in_container,lecs_to_qs,mean_user_ans_time,mean_que_ans_time,mean_question_elapsed_time,mean_question_elapsed_time_user,tag_clust
0,0,115,5692,1,1,25307.632812,False,37000.000000,False,0.695652,...,3548,0.528523,39705,1,0.000025,19933.310547,16801.380859,15473.0,20258.064453,0
1,56943,115,5716,2,1,37000.000000,False,55000.000000,False,0.695652,...,2376,0.436121,39645,4,0.000101,19933.310547,16428.082031,15253.0,20258.064453,0
2,118363,115,128,0,1,55000.000000,False,19000.000000,False,0.695652,...,1914,0.680675,39690,7,0.000176,19933.310547,18156.904297,17947.0,20258.064453,3
3,131167,115,7860,3,1,19000.000000,False,11000.000000,False,0.695652,...,2137,0.543857,39652,3,0.000076,19933.310547,20252.582031,20070.0,20258.064453,29
4,137965,115,7922,4,1,11000.000000,False,5000.000000,False,0.695652,...,1878,0.481446,69258,4,0.000058,19933.310547,17750.400391,17697.0,20258.064453,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9804599,646228695,216747867,8805,410,1,23000.000000,True,13000.000000,True,0.744071,...,624,0.632155,5940,156,0.026263,42221.257812,20012.945312,19949.0,42727.265625,0
9804600,646321314,216747867,5708,411,0,13000.000000,True,66000.000000,True,0.744071,...,5287,0.644307,5963,136,0.022807,42221.257812,47403.847656,42117.0,42727.265625,0
9804601,646393443,216747867,5434,412,0,66000.000000,True,37000.000000,True,0.744071,...,1809,0.639624,5855,148,0.025278,42221.257812,23305.748047,22033.0,42727.265625,0
9804602,646445632,216747867,6429,413,1,37000.000000,True,16000.000000,True,0.744071,...,3855,0.615425,5783,155,0.026803,42221.257812,25951.912109,23769.0,42727.265625,0


In [34]:
train.columns

Index(['timestamp', 'user_id', 'content_id', 'task_container_id',
       'answered_correctly', 'prior_question_elapsed_time',
       'prior_question_had_explanation', 'question_elapsed_time',
       'question_had_explanation', 'mean_user_accuracy', 'questions_answered',
       'mean_que_accuracy', 'question_was_asked', 'mean_container_acc',
       'container_unit_count', 'lecs_in_container', 'lecs_to_qs',
       'mean_user_ans_time', 'mean_que_ans_time', 'mean_question_elapsed_time',
       'mean_question_elapsed_time_user', 'tag_clust'],
      dtype='object')

In [35]:
#Выбираем фичи, которые будут использоваться при обучении
features = [
    'user_id',
    'content_id',
    'task_container_id',
    'prior_question_elapsed_time',
       'prior_question_had_explanation',
            'mean_user_accuracy',
            'questions_answered',
       'mean_que_accuracy',
            'question_was_asked',
            'mean_container_acc',
       'container_unit_count',
            'lecs_in_container',
            'lecs_to_qs',
       'mean_user_ans_time',
            'mean_que_ans_time',
            'mean_question_elapsed_time',
       'mean_question_elapsed_time_user',
    'question_elapsed_time',
    'question_had_explanation',
          # 'question_had_explanation_false_count',
           # 'question_had_explanation_true_count',
    'tag_clust'
]

features

['user_id',
 'content_id',
 'task_container_id',
 'prior_question_elapsed_time',
 'prior_question_had_explanation',
 'mean_user_accuracy',
 'questions_answered',
 'mean_que_accuracy',
 'question_was_asked',
 'mean_container_acc',
 'container_unit_count',
 'lecs_in_container',
 'lecs_to_qs',
 'mean_user_ans_time',
 'mean_que_ans_time',
 'mean_question_elapsed_time',
 'mean_question_elapsed_time_user',
 'question_elapsed_time',
 'question_had_explanation',
 'tag_clust']

In [36]:
print(train.dtypes)

timestamp                            int64
user_id                              int32
content_id                           int16
task_container_id                    int16
answered_correctly                   int64
prior_question_elapsed_time        float32
prior_question_had_explanation        bool
question_elapsed_time              float32
question_had_explanation           boolean
mean_user_accuracy                 float64
questions_answered                   int64
mean_que_accuracy                  float64
question_was_asked                   int64
mean_container_acc                 float64
container_unit_count                 int64
lecs_in_container                    int64
lecs_to_qs                         float64
mean_user_ans_time                 float32
mean_que_ans_time                  float32
mean_question_elapsed_time         float64
mean_question_elapsed_time_user    float32
tag_clust                            int32
dtype: object


In [37]:
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype(bool)
train['question_had_explanation'] = train['question_had_explanation'].astype(bool)

In [38]:
#Разделяем выборку на тренировочную и валидационную
train_df, test_df, y_train, y_test = train_test_split(train[features], train[target], random_state=300, test_size=0.2)

In [39]:
#чистим память
import gc
del train
gc.collect()

140

In [40]:
#only for submit
#train_df = train[features]
#y_train = train[target]


In [41]:
import xgboost
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score


XGBmodel = xgboost.XGBClassifier()
XGBmodel.fit(train_df, y_train)
# %%
print(XGBmodel)


# %%
# Делаем предикты с помощью XGB модели
XGBpred = XGBmodel.predict(test_df)
XGBpredictions = [round(value) for value in XGBpred]


# %%
# Оценка ROC-AUC кривой
print('XGBoost ROC-AUC score: ', roc_auc_score(y_test.values, XGBmodel.predict_proba(test_df)[:, 1]))

[15:04:57] WARNING: ../src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='approx', validate_parameters=1, verbosity=None)
XGBoost ROC-AUC score:  0.7781895984967472


In [42]:

del train_df
gc.collect()

15

In [43]:
## SUBMISSION
env = riiideducation.make_env()


In [44]:
iter_test = env.iter_test()

In [45]:
for (test_df, sample_prediction_df) in iter_test:
    # merge
    
    test_df = test_df.merge(user_ans_df, how='left', on='user_id')
    test_df = test_df.merge(content_answers_df, how='left', on='content_id')
    test_df = test_df.merge(container_df, how='left', on='task_container_id')
    test_df = test_df.merge(user_ans_time_df, how='left', on='user_id')
    test_df = test_df.merge(que_ans_time_df, how='left', on='content_id')
    test_df = test_df.merge(quest_well_df, how='left', on='content_id')
    test_df = test_df.merge(user_well_df, how='left', on='user_id')
    test_df = test_df.merge(questions, how='left', on = 'content_id')
    
    # type transformation
    test_df['question_elapsed_time'] = test_df['prior_question_elapsed_time'].shift(-1)

    test_df['question_had_explanation'] = test_df['prior_question_had_explanation'].shift(-1)
    
    #test_df['question_had_explanation_true_count'] = test_df['question_had_explanation_true_count'].fillna(0).astype(int)
    #test_df['question_had_explanation_false_count'] = test_df['question_had_explanation_false_count'].fillna(0).astype(int)
    #test_df['prior_question_elapsed_time'].fillna(test_df['prior_question_elapsed_time'].mean(), inplace = True)
    
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value=False).astype(bool)
    test_df['question_had_explanation'] = test_df['question_had_explanation'].fillna(value=False).astype(bool)
    test_df.replace(False, 0, inplace=True)
    test_df.replace(True, 1, inplace=True)
    
    
    #test_df['prior_question_elapsed_time'].fillna(value = test_df['prior_question_elapsed_time'].mean(), inplace = True)
    #test_df['prior_question_had_explanation'].fillna(value = test_df['prior_question_had_explanation'].mean(), inplace = True)
    #test_df['question_elapsed_time'].fillna(value = test_df['question_elapsed_time'].mean(), inplace = True)
    #test_df['mean_user_ans_time'].fillna(value = test_df['mean_user_ans_time'].mean(), inplace = True)
    #test_df['mean_que_ans_time'].fillna(value = test_df['mean_que_ans_time'].mean(), inplace = True)
    #test_df['mean_question_elapsed_time'].fillna(value = test_df['mean_question_elapsed_time'].mean(), inplace = True)
    #test_df['mean_question_elapsed_time_user'].fillna(value = test_df['mean_question_elapsed_time_user'].mean(), inplace = True)
    test_df.fillna(tr_mean, inplace = True)
    #test_df['question_elapsed_time'].fillna(test_df['question_elapsed_time'].mean(), inplace = True)
    #test_df['question_had_explanation'].fillna(value=False, inplace = True)
    #test_df['question_had_explanation'] = test_df['question_had_explanation'].astype(bool)
    #test_df.fillna(train.mean(), inplace = True)
    
    
    # preds
    test_df['answered_correctly'] = XGBmodel.predict_proba(test_df[features])[:, 1]
    cols_to_submission = ['row_id', 'answered_correctly', 'group_num']
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])